<div  style="padding: 0px; padding-left: 0px;">
<img src="lightning_logo.png" width="320px"  style="display: inline-block; margin-left: 0px;">
</div>

<p class='header-1'>Welcome to <span style='font-weight:800'>Lightning</span></p>

<p class='info-text'>Lightning is an open-source library for integrating and analyzing spatial data obtained from a vareity of imaging modalities. Lightning is capable of working with ndarrays on small, medium, and very large data sets. Like its sibling, Thunder, it exposes array operations through either local or distributed implementations with a common interface, and makes it easy to switch between them. Its distributed implementation currently targets Spark, a powerful cluster computing framework. 

<p class='info-text'>For more information on Lightning, visit the <a href='http://github.com/alexandonian/lightning'>project</a> page. 
</p>

<p class='header-2'>Tutorials</p>
These notebooks are interactive tutorials that show how Lightning works:

<a class='call-link lightning' href=>Basic usage (on its way)</a>


<br>

<p class='header-2'>Introduction</span></p>
## Rethinking the Image in Lightning:
